In [121]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse import diags
from scipy.sparse.linalg import cg
from scipy.linalg import solve
from scipy.sparse import csc_matrix
from scipy.sparse.linalg import bicg,bicgstab
from matplotlib.animation import ArtistAnimation
from matplotlib.animation import FuncAnimation, PillowWriter
from matplotlib.colors import LightSource
from matplotlib.animation import ImageMagickWriter
from matplotlib.animation import FFMpegWriter

# Описание класса для краевой задачи

In [136]:
class PDE:
    """
    Класс для решения параболических дифференциальных уравнений с помощью явных и неявных численных схем.

    Этот класс предоставляет методы для реализации явной и неявной схемы для решения 
    парциальных дифференциальных уравнений (ПДУ) с заданными краевыми условиями и источниками.

    Атрибуты:
    -----------
    f : callable
        Функция источника (или правой части) уравнения ПДУ.
    v1 : callable
        Функция начальных условий для решения.
    v2 : callable
        Функция для граничных условий в уравнении ПДУ.
    g0, g1, g2, g3 : callable
        Граничные условия для каждого из краев.
    x_bord, y_bord, t_bord : tuple
        Границы по осям x, y и времени (t) для сетки.
    explicit_right_part : lambda
        Лямбда-функция, которая вычисляет правую часть явной схемы.
    implicit_right_part : lambda
        Лямбда-функция, которая вычисляет правую часть неявной схемы.
    transition_to_next_layer : lambda
        Лямбда-функция, которая осуществляет переход к следующему слою для явной схемы.
    calc_h : lambda
        Лямбда-функция для вычисления шага по соответствующей оси.
    implicit_b_part : lambda
        Лямбда-функция для вычисления правой части матрицы в неявной схеме.

    Методы:
    --------
    get_mesh(x_cnt, y_cnt, t_cnt):
        Генерирует сетку для пространственно-временной области.
    get_ex_tenzor(x_cnt, y_cnt, t_cnt):
        Инициализирует тензор для явного решения.
    get_im_tenzor(x_cnt, y_cnt, t_cnt):
        Инициализирует тензор для неявного решения.
    get_explicit_solution(x_cnt, y_cnt, t_cnt):
        Вычисляет решение ПДУ с использованием явной схемы.
    create_matrix_DE(n, hx, hy, ht, f):
        Создает разреженную матрицу для неявной схемы.
    get_implicit_solution(x_cnt, y_cnt, t_cnt):
        Вычисляет решение ПДУ с использованием неявной схемы.
    get_dense_mesh(x_cnt, y_cnt):
        Генерирует более плотную сетку для пространственных измерений.
    start_dense_explicit():
        Запускает решение задачи с помощью явной схемы для плотной сетки.
    start_dense_implicit():
        Запускает решение задачи с помощью неявной схемы для плотной сетки.
    """
    def __init__(self,f,v1,v2,g0,g1,g2,g3,x_bord,y_bord,t_bord):
        """
        Инициализация объекта с параметрами для решения задачи с частными производными.
    
        Параметры:
        f : callable
            Функция правой части уравнения, которая зависит от тензора, времени и шагов по пространству и времени.
        v1 : float
            Значение для граничного условия по одной из осей (например, скорости).
        v2 : float
            Значение для граничного условия по другой оси (например, скорости).
        g0 : float
            Граничное условие на первой границе в пространстве.
        g1 : float
            Граничное условие на второй границе в пространстве.
        g2 : float
            Граничное условие на третьей границе в пространстве.
        g3 : float
            Граничное условие на четвертой границе в пространстве.
        x_bord : tuple
            Границы по оси x, определяющие диапазон пространственного интервала.
        y_bord : tuple
            Границы по оси y, определяющие диапазон пространственного интервала.
        t_bord : tuple
            Границы по времени, определяющие диапазон временного интервала.
    
        Атрибуты:
        explicit_right_part : lambda
            Лямбда-функция, вычисляющая правую часть для явной схемы решения.
        implicit_right_part : lambda
            Лямбда-функция, вычисляющая правую часть для неявной схемы решения.
        transition_to_next_layer : lambda
            Лямбда-функция для перехода к следующему слою решения.
        calc_h : lambda
            Лямбда-функция для расчета шага сетки по указанной оси.
        implicit_b_part : lambda
            Лямбда-функция для вычисления правой части для неявной схемы, учитывающая сдвиг по времени.
    
        """
        self.f = f
        self.v1 = v1
        self.v2 = v2
        self.g0 = g0; self.g1 = g1;  self.g2 = g2; self.g3 = g3;
        self.x_bord = x_bord; self.y_bord = y_bord; self.t_bord = t_bord;
        self.explicit_right_part = lambda tenz,t,h_x,h_y,h_t,f:(tenz[t,2:,1:-1] - 2*tenz[t,1:-1,1:-1] + tenz[t,:-2,1:-1:])/h_x**2 + (tenz[t,1:-1,2:] - 2*tenz[t,1:-1,1:-1] + tenz[t,1:-1,:-2])/h_y**2 + f
        self.implicit_right_part = lambda tenz,t,h_x,h_y,h_t,f:(tenz[t+1,2:,1:-1] - 2*tenz[t+1,1:-1,1:-1] + tenz[t+1,:-2,1:-1])/2/h_x**2 \
                                                            + (tenz[t-1,2:,1:-1] - 2*tenz[t-1,1:-1,1:-1] + tenz[t-1,:-2,1:-1])/2/h_x**2 \
                                                            + (tenz[t+1,1:-1,2:] - 2*tenz[t+1,1:-1,1:-1] + tenz[t+1,1:-1,:-2])/2/h_y**2 \
                                                            + (tenz[t-1,1:-1,2:] - 2*tenz[t-1,1:-1,1:-1] + tenz[t-1,1:-1,:-2])/2/h_y**2 + f
        self.transition_to_next_layer = lambda tenz,t,h_x,h_y,h_t,f: self.explicit_right_part(tenz,t,h_x,h_y,h_t,f)*h_t**2+2*tenz[t,1:-1,1:-1] - tenz[t-1,1:-1,1:-1]
        self.calc_h = lambda cnt, bord: (bord[1]-bord[0])/(cnt-1)
        self.implicit_b_part = lambda tenz,t,h_x,h_y,h_t,f: h_t**2*((tenz[t-1,2:,1:-1] - 2*tenz[t-1,1:-1,1:-1] + tenz[t-1,:-2, 1:-1])/(2*h_x**2) + (tenz[t-1,1:-1,2:] - 2*tenz[t-1,1:-1,1:-1] + tenz[t-1,1:-1,:-2])/(2*h_y**2) + f) + 2*tenz[t,1:-1:,1:-1] - tenz[t-1,1:-1:,1:-1]
        #получение сетки
    def get_mesh(self,x_cnt,y_cnt,t_cnt):
        """
        Генерирует трехмерную сетку для пространственно-временной области задачи.
    
        Параметры:
        x_cnt : int
            Количество узлов по оси x (по пространству).
        y_cnt : int
            Количество узлов по оси y (по пространству).
        t_cnt : int
            Количество узлов по времени.
    
        Возвращаемое значение:
        tuple
            Кортеж трех массивов, представляющих сетку по времени, пространству x и пространству y.
            Массивы создаются с использованием функции `np.meshgrid`, где оси индексируются как 'ij',
            что означает, что индексы для оси времени (t) идут первыми, затем для оси x, и наконец для оси y.
    
        Примечания:
        - Метод использует диапазоны, заданные в атрибутах `t_bord`, `x_bord` и `y_bord` для создания сетки.
        - Сетка возвращается с использованием линейных интервалов для каждой оси.
    
        """
        return np.meshgrid( np.linspace(self.t_bord[0],self.t_bord[1],t_cnt), \
                           np.linspace(self.x_bord[0],self.x_bord[1],x_cnt), \
                            np.linspace(self.y_bord[0],self.y_bord[1],y_cnt), indexing = 'ij')
    #инициализация тензора нужного разменроа для хранения решения    
    def get_ex_tenzor(self,x_cnt,y_cnt,t_cnt):
        self.ex_tenz = np.zeros((t_cnt,x_cnt,y_cnt))
    def get_im_tenzor(self,x_cnt,y_cnt,t_cnt):
        self.im_tenz = np.zeros((t_cnt,x_cnt,y_cnt))
    #реализация явной схемы
    def get_explicit_solution(self,x_cnt,y_cnt,t_cnt):
        """
        Вычисляет явное решение задачи в виде тензора значений на сетке.
    
        Параметры:
        x_cnt : int
            Количество узлов по оси x (по пространству).
        y_cnt : int
            Количество узлов по оси y (по пространству).
        t_cnt : int
            Количество узлов по времени.
    
        Возвращаемое значение:
        numpy.ndarray
            Тензор размерности (t_cnt, x_cnt, y_cnt), содержащий решение задачи на сетке. 
            Значения обновляются на каждом шаге времени с использованием явного метода.
    
        Примечания:
        - Метод использует функцию `get_mesh` для получения сетки и функцию `calc_h` для вычисления шага по каждой из осей.
        - На первом шаге времени значения в тензоре и на границах задаются с использованием граничных условий (`g0`, `g1`, `g2`, `g3` и начального условия `v1`).
        - Для каждого последующего шага времени вычисляется новое значение на основе предыдущего шага с использованием явного метода и функции правой части уравнения (`explicit_right_part`).
        - Результирующий тензор содержит решение на каждом шаге времени для всех точек сетки.
    
        """
        self.get_ex_tenzor(x_cnt,y_cnt,t_cnt)
        t_,x_,y_ = self.get_mesh(x_cnt,y_cnt,t_cnt)
        h_x = self.calc_h(x_cnt,self.x_bord)
        h_y = self.calc_h(y_cnt,self.y_bord)
        h_t = self.calc_h(t_cnt,self.t_bord)
        
        self.ex_tenz [0,:,:] = self.v1(x_[0,:,:],y_[0,:,:])
        self.ex_tenz[:,0,:] = self.g0(x_[:,0,:],y_[:,0,:],t_[:,0,:])
        self.ex_tenz[:,-1,:] = self.g1(x_[:,-1,:],y_[:,-1,:],t_[:,-1,:])
        self.ex_tenz[:,:,0] = self.g2(x_[:,:,0],y_[:,:,0],t_[:,:,0])
        self.ex_tenz[:,:,-1] = self.g3(x_[:,:,-1],y_[:,:,-1],t_[:,:,-1])
    
        self.ex_tenz[1,1:-1,1:-1] = self.v1(x_[0,1:-1,1:-1],y_[0,1:-1,1:-1]) + h_t*self.v2(x_[0,1:-1,1:-1],y_[0,1:-1,1:-1]) + (h_t**2)/2*self.explicit_right_part(self.ex_tenz,0,h_x,h_y,h_t,f(x_[0,1:-1,1:-1],y_[0,1:-1,1:-1],t_[0,1:-1,1:-1]))
    
        for i in range(2,t_cnt):
            self.ex_tenz[i,1:-1,1:-1] = self.transition_to_next_layer(self.ex_tenz,i-1,h_x,h_y,h_t,self.f(x_[i-1,1:-1,1:-1],y_[i-1,1:-1,1:-1],t_[i-1,1:-1,1:-1]))
    
        return self.ex_tenz


    #методы для НЕявной схемы
    def create_matrix_DE(self,n,hx,hy,ht,f):
        """
        Создает матрицу системы линейных уравнений (СЛАУ) и правую часть для численного решения уравнений в частных производных.
    
        Параметры:
        n : int
            Номер временного слоя, для которого вычисляется матрица и правая часть.
        hx : float
            Шаг сетки по оси x.
        hy : float
            Шаг сетки по оси y.
        ht : float
            Шаг сетки по времени.
        f : callable
            Функция правой части уравнения, принимающая координаты x, y и время t.
    
        Возвращаемое значение:
        tuple
            A_ : scipy.sparse.csc_matrix
                Разреженная матрица коэффициентов СЛАУ (размерностью N x N, где N = m * k).
            b : numpy.ndarray
                Вектор правой части системы (размерностью N).
    
        Описание:
        - Метод формирует разреженную матрицу A_ для неявного численного метода решения задачи с использованием схемы с центральными разностями.
        - Матрица A_ включает главную диагональ и дополнительные диагонали, соответствующие численной аппроксимации второго порядка для пространственных производных.
        - Вектор b рассчитывается с учетом граничных условий и правой части уравнения. Он представляет собой преобразованное значение функции `implicit_b_part`.
    
        Примечания:
        - Корректировка элементов матрицы A_ и вектора b осуществляется для учета граничных условий задачи.
        - Возвращаемый вектор b преобразуется в одномерный массив для последующего решения системы.
    
        """
        m = self.im_tenz.shape[1] - 2
        k = self.im_tenz.shape[2] - 2
        
        N=m*k
        diagonals = [
            (1 + ht**2/hx**2 + ht**2/hy**2) * np.ones(N),      # главная диагональ (4)
            (-1)*ht**2/(2*hx**2) * np.ones(N - 1), # нижняя диагональ (-1)
            (-1)*ht**2/(2*hx**2) * np.ones(N - 1), # верхняя диагональ (-1)
            (-1)*ht**2/(2*hy**2) * np.ones(N - m), # нижняя диагональ (-m)
            (-1)*ht**2/(2*hy**2)* np.ones(N - m), # верхняя диагональ (+m)
        ]
        
        diagonals[1][m-1::m] = 0
        diagonals[2][m-1::m] = 0
        diagonals[3][:m+1] = 0
        diagonals[4][-m] = 0
    
        # Создание разреженной матрицы A
        A_ = csc_matrix(diags(diagonals, offsets=[0, -1, 1,-m,m], shape=(N, N)))
        
        
        
        
        b = self.implicit_b_part(self.im_tenz,n-1,hx,hy,ht,f)
    #     print('b: ',(b>0).any())
        b[0,0] = b[0,0] + self.im_tenz[n,0,1] * (ht**2)/(2*hx**2) + self.im_tenz[n,1,0] * (ht**2)/(2*hy**2)
        b[0,-1] = b[0,-1] + self.im_tenz[n,0,-2] * (ht**2)/(2*hx**2) + self.im_tenz[n,1,-1] * (ht**2)/(2*hy**2)
        b[-1,0] = b[-1,0] + self.im_tenz[n,-1,1] * (ht**2)/(2*hx**2) + self.im_tenz[n,-2,0] * (ht**2)/(2*hy**2)
        b[-1,-1] = b[-1,-1] + self.im_tenz[n,-1,-2] * (ht**2)/(2*hx**2) + self.im_tenz[n,-2,-1] * (ht**2)/(2*hy**2)
        b[0,1:-1] = b[0,1:-1] + self.im_tenz[n, 0, 2:-2] *  (ht**2)/(2*hx**2)
        b[-1,1:-1] = b[-1,1:-1] + self.im_tenz[n, -1, 2:-2] *  (ht**2)/(2*hx**2)
        b[1:-1,0] = b[1:-1,0] + self.im_tenz[n,2:-2, 0] *  (ht**2)/(2*hy**2)
        b[1:-1,-1] = b[1:-1,-1] + self.im_tenz[n,2:-2, -1] *  (ht**2)/(2*hy**2)
           
        return A_,b.reshape(m*k)
    
    def get_implicit_solution(self,x_cnt,y_cnt,t_cnt):
        """
        Вычисляет численное решение задачи с использованием неявного метода.
    
        Параметры:
        x_cnt : int
            Количество узлов сетки по оси x.
        y_cnt : int
            Количество узлов сетки по оси y.
        t_cnt : int
            Количество временных слоев.
    
        Возвращаемое значение:
        numpy.ndarray
            Трехмерный тензор `im_tenz` размерностью (t_cnt, x_cnt, y_cnt), содержащий значения решения в узлах сетки.
    
        Описание:
        - Метод решает задачу с использованием неявной схемы для численного решения уравнений в частных производных.
        - На первом временном слое используется явная схема для вычисления начальных значений.
        - На каждом последующем временном слое формируется система линейных алгебраических уравнений (СЛАУ) с помощью метода `create_matrix_DE`.
        - Решение СЛАУ выполняется с использованием метода бисопряженных градиентов (bicgstab).
    
        Алгоритм:
        1. Заполняются граничные и начальные условия для тензора `im_tenz`.
        2. Для каждого временного слоя создается матрица системы A и вектор правой части b.
        3. Решается система Ax = b для вычисления значений решения на текущем временном слое.
        4. Проверяется сходимость решения методом `bicgstab`. Если метод не сходится, выводится сообщение с кодом завершения.
    
        Примечания:
        - Метод выводит промежуточные сообщения каждые 10 итераций для контроля прогресса.
        - Решение возвращается в виде трехмерного тензора, где каждая ось соответствует времени, x и y соответственно.
    
        """
        self.get_im_tenzor(x_cnt,y_cnt,t_cnt)
        t_,x_,y_ = self.get_mesh(x_cnt,y_cnt,t_cnt)
        h_x = self.calc_h(x_cnt,self.x_bord)
        h_y = self.calc_h(y_cnt,self.y_bord)
        h_t = self.calc_h(t_cnt,self.t_bord)
        
        self.im_tenz [0,:,:] = self.v1(x_[0,:,:],y_[0,:,:])
        self.im_tenz[:,0,:] = self.g0(x_[:,0,:],y_[:,0,:],t_[:,0,:])
        self.im_tenz[:,-1,:] = self.g1(x_[:,-1,:],y_[:,-1,:],t_[:,-1,:])
        self.im_tenz[:,:,0] = self.g2(x_[:,:,0],y_[:,:,0],t_[:,:,0])
        self.im_tenz[:,:,-1] = self.g3(x_[:,:,-1],y_[:,:,-1],t_[:,:,-1])
    
        self.im_tenz[1,1:-1,1:-1] = self.v1(x_[0,1:-1,1:-1],y_[0,1:-1,1:-1]) + h_t*self.v2(x_[0,1:-1,1:-1],y_[0,1:-1,1:-1]) + (h_t**2)/2*self.explicit_right_part(self.im_tenz,0,h_x,h_y,h_t,self.f(x_[0,1:-1,1:-1],y_[0,1:-1,1:-1],t_[0,1:-1,1:-1]))
    
        for i in range(2,t_cnt):
            A,b = self.create_matrix_DE(i,h_x,h_y,h_t,self.f(x_[i,1:-1,1:-1],y_[i,1:-1,1:-1],t_[i,1:-1,1:-1]))
        #     b_s.append(b)
        #     x, info = cg(A, b,tol=1e-12)
        
            # Используем метод бисопряженных градиентов
            x, info = bicgstab(A, b)
            self.im_tenz[i,1:-1,1:-1] = np.copy(x.reshape((x_cnt-2,y_cnt-2)))
            if info != 0:
                print("Метод не сошелся. Код завершения:", info)
            if(i%10==0):
                print('Ура! Уже ', i, ' итераций!')
    
        return self.im_tenz

    def get_dense_mesh(self,x_cnt,y_cnt):
        return np.meshgrid(np.linspace(self.x_bord[0],self.x_bord[1],x_cnt), \
                            np.linspace(self.y_bord[0],self.y_bord[1],y_cnt), indexing = 'ij')
    def get_ex_tenzor_dense(self,x_cnt,y_cnt,t_cnt):
        self.ex_tenz_d = np.zeros((3,x_cnt,y_cnt))
    def get_im_tenzor_dense(self,x_cnt,y_cnt,t_cnt):
        self.im_tenz_d = np.zeros((3,x_cnt,y_cnt))

    def start_dense_explicit(self):
        self.tex_dense_curr = self.t_bord[0]
        self.get_ex_tenzor(x_cnt,y_cnt,t_cnt)
        x,y = self.get_mesh(x_cnt,y_cnt)
        t = np.linspace(self.t_bord[0],self.t_bord[1],t_cnt)
        h_x = self.calc_h(x_cnt,self.x_bord)
        h_y = self.calc_h(y_cnt,self.y_bord)
        h_t = self.calc_h(t_cnt,self.t_bord)
        
        self.ex_tenz_d[0,:,:] = self.v1(x,y)
        self.ex_tenz_d[0,:] = self.g0(x[0,:],y[0,:],t[0])
        self.ex_tenz_d[-1,:] = self.g1(x[-1,:],y[-1,:],t[-1])
        self.ex_tenz_d[:,0] = self.g2(x[:,0],y[:,0],t[:,0])
        self.ex_tenz_d[:,-1] = self.g3(x[:,-1],y[:,-1],t[-1])
    
        self.ex_tenz_d[1:-1,1:-1] = self.v1(x[1:-1,1:-1],y[1:-1,1:-1]) + h_t*self.v2(x[1:-1,1:-1],y[1:-1,1:-1]) + (h_t**2)/2*self.explicit_right_part(self.ex_tenz,0,h_x,h_y,h_t,f(x[1:-1,1:-1],y[1:-1,1:-1],t[0]*np.ones(x[1:-1,1:-1].shape)))
    
    def start_dense_implicit(self):
        self.tim_dense_curr = self.t_bord[0]
        x,y = self.get_mesh(x_cnt,y_cnt)
        t = np.linspace(self.t_bord[0],self.t_bord[1],t_cnt)
        h_x = self.calc_h(x_cnt,self.x_bord)
        h_y = self.calc_h(y_cnt,self.y_bord)
        h_t = self.calc_h(t_cnt,self.t_bord)
        
        self.im_tenz [0,:,:] = self.v1(x_[0,:,:],y_[0,:,:])
        self.im_tenz[:,0,:] = self.g0(x_[:,0,:],y_[:,0,:],t_[:,0,:])
        self.im_tenz[:,-1,:] = self.g1(x_[:,-1,:],y_[:,-1,:],t_[:,-1,:])
        self.im_tenz[:,:,0] = self.g2(x_[:,:,0],y_[:,:,0],t_[:,:,0])
        self.im_tenz[:,:,-1] = self.g3(x_[:,:,-1],y_[:,:,-1],t_[:,:,-1])
    
        self.im_tenz[1,1:-1,1:-1] = self.v1(x_[0,1:-1,1:-1],y_[0,1:-1,1:-1]) + h_t*self.v2(x_[0,1:-1,1:-1],y_[0,1:-1,1:-1]) + (h_t**2)/2*self.explicit_right_part(self.im_tenz,0,h_x,h_y,h_t,self.f(x_[0,1:-1,1:-1],y_[0,1:-1,1:-1],t_[0,1:-1,1:-1]))

# Класс для анимации

In [160]:
class PDESolutionAnimator:
    """
    Класс для анимации решения уравнений в частных производных (УЧП) в различных форматах,
    таких как GIF и MP4 видео. Анимации визуализируют данные решения через 2D и 3D графики,
    используя эффект "hillshading" для данных высот, поверхность и сравнение между неявными и явными методами.

    Атрибуты:
    ----------
    PDEname : str
        Название решаемого УЧП, используется для именования выходных файлов.
    ls : LightSource
        Экземпляр класса LightSource, используемый для наложения эффекта hillshading на данные решения.
    cmap : matplotlib.colors.Colormap
        Колор карта, используемая для визуализаций (по умолчанию: серый).
    ve : float
        Коэффициент вертикального увеличения для 3D визуализаций (по умолчанию: 0.05).

    Методы:
    --------
    get_cmap_gif_v1(solution, out_FPS, gif_name='default.gif'):
        Создает GIF, показывающий эволюцию решения УЧП с использованием эффекта hillshading в серых тонах.
    
    get_cmap_gif_v2(solution, out_FPS, gif_name='default.gif'):
        Создает GIF, показывающий эволюцию решения УЧП с наложением оттенков в серых тонах.
    
    get_surface_gif_v1(solution, x, y, out_FPS, gif_name='default.gif'):
        Создает 3D GIF, визуализирующий решение УЧП по времени.
    
    comparation_gif_v1(implicit_solution, explicit_solution, x, y, t, out_FPS, gif_name='default.gif'):
        Создает GIF, сравнивающий неявное и явное решения с использованием 3D рассеяния и 2D изображений.
    
    comparation_gif_v2(implicit_solution, explicit_solution, x, y, t, out_FPS, gif_name='default.gif'):
        Создает GIF, сравнивающий неявное и явное решения с динамической настройкой диапазона и подписанными заголовками.
    
    comparation_mp4(implicit_solution, explicit_solution, x, y, t, out_FPS, mp4_name='default.mp4'):
        Создает MP4 видео, сравнивающее неявное и явное решения с улучшенной интерактивностью и обновлением кадров.
    """
    def __init__(self,PDEname,PDEchar):
        self.PDEname=PDEname
        self.PDEchar=PDEchar
        self.ls = LightSource(azdeg=315, altdeg=45)
        self.cmap = plt.cm.gray
        self.ve=0.05
    def get_cmap_gif_v1(self,solution, out_FPS, gif_name = 'default.gif'):
        """
        Создает GIF-анимацию на основе заданного 3D-решения задачи.
    
        Параметры:
        solution : numpy.ndarray
            Тензор решения задачи, где первая ось соответствует времени, а остальные две — пространственным координатам.
        out_FPS : int
            Частота кадров для выходного GIF-файла.
        gif_name : str, optional
            Имя выходного GIF-файла. По умолчанию 'default.gif'.
    
        Атрибуты:
        PDEname : str
            Название решаемого дифференциального уравнения, используется в имени выходного файла.
        ls : matplotlib.colors.LightSource
            Источник света, используемый для визуализации рельефа решений.
        cmap : matplotlib.colors.Colormap
            Цветовая карта для визуализации.
        ve : float
            Коэффициент вертикального масштабирования для рельефа.
    
        Описание:
        Создает GIF-анимацию, где каждый кадр представляет hillshade-визуализацию одного слоя тензора решения `solution`. 
        Анимация сохраняется с заданной частотой кадров `out_FPS` в файл с именем, включающим `PDEname` и `gif_name`.
        """
        fig = plt.figure()
        ax = fig.add_subplot()
        frames = []
        for p in range(solution.shape[0]):
            img = ax.imshow(self.ls.hillshade(solution[p], vert_exag=self.ve), cmap=self.cmap)
            frames.append([img])
        animation = ArtistAnimation(
            fig,                # фигура, где отображается анимация
            frames,              # кадры
            interval=1000/12,        # задержка между кадрами в мс
            blit=True,          # использовать ли двойную буферизацию
            repeat=True)       # зацикливать ли анимацию
        
        animation.save(self.PDEname+gif_name, writer = ImageMagickWriter(fps=out_FPS))

    def get_cmap_gif_v2(self,solution, out_FPS, gif_name = 'default.gif'):
        """
        Создает GIF-анимацию с обновлением изображения на основе заданного 3D-решения задачи.
    
        Параметры:
        solution : numpy.ndarray
            Тензор решения задачи, где первая ось соответствует времени, а остальные две — пространственным координатам.
        out_FPS : int
            Частота кадров для выходного GIF-файла.
        gif_name : str, optional
            Имя выходного GIF-файла. По умолчанию 'default.gif'.
    
        Атрибуты:
        PDEname : str
            Название решаемого дифференциального уравнения, используется в имени выходного файла.
        ls : matplotlib.colors.LightSource
            Источник света, используемый для визуализации рельефа решений.
        cmap : matplotlib.colors.Colormap
            Цветовая карта для визуализации.
        ve : float
            Коэффициент вертикального масштабирования для рельефа.
    
        Описание:
        Создает GIF-анимацию, где каждый кадр представляет hillshade-визуализацию одного слоя тензора решения `solution`.
        Анимация реализована с помощью `FuncAnimation`, которая обновляет существующий объект `imshow` для повышения производительности.
        Анимация сохраняется с заданной частотой кадров `out_FPS` в файл с именем, включающим `PDEname` и `gif_name`.
        """
        Z = self.ls.shade(solution[0], cmap=self.cmap, vert_exag=self.ve, blend_mode='overlay')
        fig, ax = plt.subplots()
        im = ax.imshow(Z)
        
        def update(frame):
            Z = self.ls.shade(solution[frame], cmap=self.cmap, vert_exag=self.ve, blend_mode='overlay')
            im.set_array(Z)  # обновляем значения Z
            # im.set_clim(Z.min(),Z.max())
            return [im]
        animation = FuncAnimation(fig, update, frames=solution.shape[0], interval=1000/12, blit=False)
        animation.save(self.PDEname+gif_name, writer = ImageMagickWriter(fps=out_FPS))
        
    def get_surface_gif_v1(self,solution,x,y, out_FPS, gif_name = 'default.gif'):
        """
        Создает GIF-анимацию с поверхностной 3D-визуализацией решения задачи.
    
        Параметры:
        solution : numpy.ndarray
            Тензор решения задачи, где первая ось соответствует времени, а оставшиеся — пространственным координатам.
        x : numpy.ndarray
            Координаты по оси X для визуализации.
        y : numpy.ndarray
            Координаты по оси Y для визуализации.
        out_FPS : int
            Частота кадров для выходного GIF-файла.
        gif_name : str, optional
            Имя выходного GIF-файла. По умолчанию 'default.gif'.
    
        Атрибуты:
        PDEname : str
            Название решаемого дифференциального уравнения, используется в имени выходного файла.
    
        Описание:
        Создает GIF-анимацию, где каждый кадр представляет 3D-визуализацию решения задачи в виде цветного рассеяния точек.
        Анимация сохраняется с заданной частотой кадров `out_FPS` в файл с именем, включающим `PDEname` и `gif_name`.
        """
        fig = plt.figure()
        ax_3d = fig.add_subplot(projection='3d')
        frames = []
        for p in range(solution.shape[0]):
            surface = ax_3d.scatter(x, y, solution[p], cmap='inferno',c=solution[p])
            frames.append([surface])
        animation = ArtistAnimation(
            fig,                # фигура, где отображается анимация
            frames,              # кадры
            interval=1000/12,        # задержка между кадрами в мс
            blit=True,          # использовать ли двойную буферизацию
            repeat=True)       # зацикливать ли анимацию
        
        animation.save(self.PDEname+gif_name, writer = ImageMagickWriter(fps=out_FPS))

    def comparation_gif_v1(self,implicit_solution,explicit_solution,x,y,t, out_FPS,gif_name ='default.gif'):
        """
        Создает GIF-анимацию для сравнения двух решений задачи (явного и неявного) на различных графиках.
    
        Параметры:
        implicit_solution : numpy.ndarray
            Решение задачи с использованием неявного метода.
        explicit_solution : numpy.ndarray
            Решение задачи с использованием явного метода.
        x : numpy.ndarray
            Координаты по оси X для визуализации.
        y : numpy.ndarray
            Координаты по оси Y для визуализации.
        t : numpy.ndarray
            Время (или другие параметры), используемые для отображения в заголовке.
        out_FPS : int
            Частота кадров для выходного GIF-файла.
        gif_name : str, optional
            Имя выходного GIF-файла. По умолчанию 'default.gif'.
    
        Атрибуты:
        PDEname : str
            Название решаемого дифференциального уравнения, используется в имени выходного файла.
    
        Описание:
        Создает GIF-анимацию, на которой сравниваются два метода решения (явный и неявный) на 4 графиках:
        1. 3D-график для неявного решения.
        2. 2D-график для неявного решения.
        3. 3D-график для явного решения.
        4. 2D-график для явного решения.
        Время отображается в заголовке графиков.
        Анимация сохраняется с заданной частотой кадров `out_FPS` в файл с именем, включающим `PDEname` и `gif_name`.
        """
        fig = plt.figure(figsize=(10,10))
        fig.suptitle(self.PDEname)
        ax1 = fig.add_subplot(221, projection = '3d')
        ax2 = fig.add_subplot(222)
        ax3 = fig.add_subplot(223, projection = '3d')
        ax4 = fig.add_subplot(224)
    
        frames = []
        title1 = ax1.text(0.5, 1.05,0, s=f'Время: {t[0, 0, 0]:.4f}', transform=ax1.transAxes, ha='center', va='bottom')
        title2 = ax3.text(0.5, 1.05,0, s=f'Время: {t[0, 0, 0]:.4f}', transform=ax3.transAxes, ha='center', va='bottom')
        for p in range(implicit_solution.shape[0]):
            

            Z_im = self.ls.shade(implicit_solution[p],cmap=self.cmap,vert_exag=self.ve, blend_mode='overlay')
            Z_ex = self.ls.shade(explicit_solution[p],cmap=self.cmap,vert_exag=self.ve, blend_mode='overlay')
            line2 = ax2.imshow(Z_im, animated=True)
            line4 = ax4.imshow(Z_ex, animated=True)
            line1 = ax1.scatter(x, y, implicit_solution[p], cmap='inferno',c=implicit_solution[p], animated=True)
            line3 = ax3.scatter(x, y, explicit_solution[p], cmap='inferno',c=explicit_solution[p], animated=True)
            title1.set_text(f'Время: {t[p, 0, 0]:.4f}')
            title2.set_text(f'Время: {t[p, 0, 0]:.4f}')
            frames.append([line1,line2,line3,line4,title1,title2])

        animation = ArtistAnimation(
            fig,                # фигура, где отображается анимация
            frames,              # кадры
            interval=1000/12,        # задержка между кадрами в мс
            blit=False,          # использовать ли двойную буферизацию
            repeat=True)       # зацикливать ли анимацию
        
        animation.save(self.PDEname+gif_name, writer = ImageMagickWriter(fps=out_FPS))

    def comparation_gif_v2(self,implicit_solution,explicit_solution,x,y,t, out_FPS,gif_name ='default.gif'):
        """
        Создает GIF-анимацию для сравнения двух решений задачи (явного и неявного) на различных графиках.
        В отличие от v1, эта версия обновляет значения изображений и графиков с использованием `FuncAnimation`.
    
        Параметры:
        implicit_solution : numpy.ndarray
            Решение задачи с использованием неявного метода.
        explicit_solution : numpy.ndarray
            Решение задачи с использованием явного метода.
        x : numpy.ndarray
            Координаты по оси X для визуализации.
        y : numpy.ndarray
            Координаты по оси Y для визуализации.
        t : numpy.ndarray
            Время (или другие параметры), используемые для отображения в заголовке.
        out_FPS : int
            Частота кадров для выходного GIF-файла.
        gif_name : str, optional
            Имя выходного GIF-файла. По умолчанию 'default.gif'.
    
        Атрибуты:
        PDEname : str
            Название решаемого дифференциального уравнения, используется в имени выходного файла.
    
        Описание:
        Создает GIF-анимацию, на которой сравниваются два метода решения (явный и неявный) на 4 графиках:
        1. 3D-график для неявного решения.
        2. 2D-график для неявного решения.
        3. 3D-график для явного решения.
        4. 2D-график для явного решения.
        Время отображается в заголовке графиков.
        Анимация сохраняется с заданной частотой кадров `out_FPS` в файл с именем, включающим `PDEname` и `gif_name`.
        """
        fig = plt.figure(figsize=(10, 10))
        fig.suptitle(self.PDEname)
        ax1 = fig.add_subplot(221, projection='3d')
        ax2 = fig.add_subplot(222)
        ax3 = fig.add_subplot(223, projection='3d')
        ax4 = fig.add_subplot(224)
    
        Z_im = self.ls.shade(implicit_solution[0],cmap=self.cmap, vert_exag=self.ve, blend_mode='overlay')
        Z_ex = self.ls.shade(explicit_solution[0],cmap=self.cmap, vert_exag=self.ve, blend_mode='overlay')
    
        zim_min = implicit_solution.min()
        zim_max = implicit_solution.max()
        zex_min = explicit_solution.min()
        zex_max = explicit_solution.max()
    
        z1_max = np.abs(implicit_solution).max()
        z2_max = np.abs(explicit_solution).max()
        ax1.set_zlim([-2*z1_max,2*z1_max])
        ax3.set_zlim([-2*z2_max,2*z2_max])
        line1 = ax1.scatter(x, y, implicit_solution[0], cmap='inferno',c=implicit_solution[0],vmin=zim_min, vmax = zim_max)
        line2 = ax2.imshow(Z_im, cmap=self.cmap)
        line3 = ax3.scatter(x, y, explicit_solution[0], cmap='inferno',c=explicit_solution[0] ,vmin=zex_min, vmax = zex_max)
        line4 = ax4.imshow(Z_ex, cmap=self.cmap)
        fig.colorbar(line1,ax=ax1,fraction =0.05,pad=0.05)
        fig.colorbar(line3,ax=ax3,fraction =0.05,pad=0.05)
        
        # Функция обновления кадров
        def update(frame):
    
            for collection in ax1.collections:
                collection.remove()
            for collection in ax3.collections:
                collection.remove()
            for collection in ax2.collections:
                collection.remove()
            
            Z_im = self.ls.shade(implicit_solution[frame],cmap=self.cmap, vert_exag=self.ve, blend_mode='overlay')
            Z_ex = self.ls.shade(explicit_solution[frame],cmap=self.cmap, vert_exag=self.ve, blend_mode='overlay')
            # Z_im = implicit_solution[frame]
            # Z_ex = explicit_solution[frame]
            line2.set_array(Z_im)
            line4.set_array(Z_ex)
            line2.set_clim(Z_im.min(), Z_im.max())
            line4.set_clim(Z_ex.min(), Z_ex.max())
    
            line1 = ax1.scatter(x, y, implicit_solution[frame], cmap='inferno',c=implicit_solution[frame],vmin=zim_min, vmax = zim_max)
            line3 = ax3.scatter(x, y, explicit_solution[frame], cmap='inferno',c=explicit_solution[frame],vmin=zex_min, vmax = zex_max)
            
            ax2.set_title(f'Неявный метод \n Время: {t[frame,0,0]:.4f}')
            ax4.set_title(f'Явный метод \n Время: {t[frame,0,0]:.4f}')
            ax1.set_title(f'Неявный метод \n Время: {t[frame,0,0]:.4f}')
            ax3.set_title(f'Явный метод \n Время: {t[frame,0,0]:.4f}')
        
            return line1,line2,line3,line4
    
        plt.subplots_adjust(hspace=0.5, wspace=0.75)
        # Создаем анимацию
        animation = FuncAnimation(fig, update, frames=implicit_solution.shape[0], interval=1000/12, blit=True) 
        # Сохраняем как GIF
        animation.save(self.PDEname+gif_name, writer = ImageMagickWriter(fps=out_FPS))

    def comparation_mp4(self,implicit_solution,explicit_solution,x,y,t, out_FPS,mp4_name ='default.mp4'):
        """
        Создает видеофайл в формате MP4 для сравнения двух решений задачи (явного и неявного) на различных графиках.
    
        Параметры:
        implicit_solution : numpy.ndarray
            Решение задачи с использованием неявного метода.
        explicit_solution : numpy.ndarray
            Решение задачи с использованием явного метода.
        x : numpy.ndarray
            Координаты по оси X для визуализации.
        y : numpy.ndarray
            Координаты по оси Y для визуализации.
        t : numpy.ndarray
            Время (или другие параметры), используемые для отображения в заголовке.
        out_FPS : int
            Частота кадров для выходного видеофайла.
        mp4_name : str, optional
            Имя выходного MP4 файла. По умолчанию 'default.mp4'.
    
        Атрибуты:
        PDEname : str
            Название решаемого дифференциального уравнения, используется в имени выходного файла.
    
        Описание:
        Создает видеофайл в формате MP4, на котором сравниваются два метода решения (явный и неявный) на 4 графиках:
        1. 3D-график для неявного решения.
        2. 2D-график для неявного решения.
        3. 3D-график для явного решения.
        4. 2D-график для явного решения.
        Время отображается в заголовке графиков.
        Видео сохраняется в файл с заданной частотой кадров `out_FPS` в формате MP4 с именем, включающим `PDEname` и `mp4_name`.
        """
        fig = plt.figure(figsize=(10, 10))
        fig.suptitle(self.PDEname + self.PDEchar)
        ax1 = fig.add_subplot(221, projection='3d')
        ax2 = fig.add_subplot(222)
        ax3 = fig.add_subplot(223, projection='3d')
        ax4 = fig.add_subplot(224)
    
        Z_im = self.ls.shade(implicit_solution[0],cmap=self.cmap, vert_exag=self.ve, blend_mode='overlay')
        Z_ex = self.ls.shade(explicit_solution[0],cmap=self.cmap, vert_exag=self.ve, blend_mode='overlay')
    
        zim_min = implicit_solution.min()
        zim_max = implicit_solution.max()
        zex_min = explicit_solution.min()
        zex_max = explicit_solution.max()
    
        z1_max = np.abs(implicit_solution).max()
        z2_max = np.abs(explicit_solution).max()
        ax1.set_zlim([-2*z1_max,2*z1_max])
        ax3.set_zlim([-2*z2_max,2*z2_max])
        line1 = ax1.scatter(x, y, implicit_solution[0], cmap='inferno',c=implicit_solution[0],vmin=zim_min, vmax = zim_max)
        line2 = ax2.imshow(Z_im, cmap=self.cmap)
        line3 = ax3.scatter(x, y, explicit_solution[0], cmap='inferno',c=explicit_solution[0] ,vmin=zex_min, vmax = zex_max)
        line4 = ax4.imshow(Z_ex, cmap=self.cmap)
        fig.colorbar(line1,ax=ax1,fraction =0.05,pad=0.05)
        fig.colorbar(line3,ax=ax3,fraction =0.05,pad=0.05)
        
        # Функция обновления кадров
        def update(frame):
    
            for collection in ax1.collections:
                collection.remove()
            for collection in ax3.collections:
                collection.remove()
            for collection in ax2.collections:
                collection.remove()
            
            Z_im = self.ls.shade(implicit_solution[frame],cmap=self.cmap, vert_exag=self.ve, blend_mode='overlay')
            Z_ex = self.ls.shade(explicit_solution[frame],cmap=self.cmap, vert_exag=self.ve, blend_mode='overlay')
            # Z_im = implicit_solution[frame]
            # Z_ex = explicit_solution[frame]
            line2.set_array(Z_im)
            line4.set_array(Z_ex)
            line2.set_clim(Z_im.min(), Z_im.max())
            line4.set_clim(Z_ex.min(), Z_ex.max())
    
            line1 = ax1.scatter(x, y, implicit_solution[frame], cmap='inferno',c=implicit_solution[frame],vmin=zim_min, vmax = zim_max)
            line3 = ax3.scatter(x, y, explicit_solution[frame], cmap='inferno',c=explicit_solution[frame],vmin=zex_min, vmax = zex_max)
            
            ax2.set_title(f'Неявный метод \n Время: {t[frame,0,0]:.4f}')
            ax4.set_title(f'Явный метод \n Время: {t[frame,0,0]:.4f}')
            ax1.set_title(f'Неявный метод \n Время: {t[frame,0,0]:.4f}')
            ax3.set_title(f'Явный метод \n Время: {t[frame,0,0]:.4f}')
        
            return line1,line2,line3,line4
    
        plt.subplots_adjust(hspace=0.5, wspace=0.75)
        animation = FuncAnimation(fig, update, frames=implicit_solution.shape[0], interval=150, blit=True) 
        
        FFWriter = FFMpegWriter(fps=out_FPS)
        animation.save(self.PDEname+mp4_name,writer=FFWriter,dpi =200)

# Решение модельных задач

## Модель 1

$$\begin{align}
& f = e^{(-10((x-0.5)^2 +(y-0.5^2)))-t}\\
& v_1 = 0\\
& v_2 = 0\\
& g = 0
\end{align}$$

In [174]:
f = lambda x,y,t: np.exp(-10*((x-0.5)**2 + (y-0.5)**2) - t )
g0 = lambda x,y,t: np.zeros(x.shape)
g1 = lambda x,y,t: np.zeros(x.shape)
g2 = lambda x,y,t: np.zeros(x.shape)
g3 = lambda x,y,t: np.zeros(x.shape)
v_1 = lambda x,y: np.zeros(x.shape)
v_2 = lambda x,y: np.zeros(x.shape)

x_bord = np.array([0,1]);  y_bord = np.array([0,1]);  t1_bord = np.array([0,1]); t2_bord = np.array([0,1]);
x_cnt = 100; y_cnt = 100; t_cnt = 300;
model_name = 'model_1'
model_title = r'$\begin{array}& g = 0, & v_1 = 0, & v_2 = 0, & f = e^{-10((x-0.5)^2 + (y-0.5)^2)-t}\end{array}$'
outFPS=12

In [175]:
pde1_1 = PDE(f,v_1,v_2,g0,g1,g2,g3,x_bord,y_bord,t1_bord)
pde1_2 = PDE(f,v_1,v_2,g0,g1,g2,g3,x_bord,y_bord,t2_bord)

In [176]:
explicit_solution1_1 = pde1_1.get_explicit_solution(x_cnt,y_cnt,t_cnt)
implicit_solution1_1 = pde1_1.get_implicit_solution(x_cnt,y_cnt,t_cnt)

explicit_solution1_2 = pde1_2.get_explicit_solution(x_cnt,y_cnt,t_cnt)
implicit_solution1_2 = pde1_2.get_implicit_solution(x_cnt,y_cnt,t_cnt)

Ура! Уже  10  итераций!
Ура! Уже  20  итераций!
Ура! Уже  30  итераций!
Ура! Уже  40  итераций!
Ура! Уже  50  итераций!
Ура! Уже  60  итераций!
Ура! Уже  70  итераций!
Ура! Уже  80  итераций!
Ура! Уже  90  итераций!
Ура! Уже  100  итераций!
Ура! Уже  110  итераций!
Ура! Уже  120  итераций!
Ура! Уже  130  итераций!
Ура! Уже  140  итераций!
Ура! Уже  150  итераций!
Ура! Уже  160  итераций!
Ура! Уже  170  итераций!
Ура! Уже  180  итераций!
Ура! Уже  190  итераций!
Ура! Уже  200  итераций!
Ура! Уже  210  итераций!
Ура! Уже  220  итераций!
Ура! Уже  230  итераций!
Ура! Уже  240  итераций!
Ура! Уже  250  итераций!
Ура! Уже  260  итераций!
Ура! Уже  270  итераций!
Ура! Уже  280  итераций!
Ура! Уже  290  итераций!
Ура! Уже  10  итераций!
Ура! Уже  20  итераций!
Ура! Уже  30  итераций!
Ура! Уже  40  итераций!
Ура! Уже  50  итераций!
Ура! Уже  60  итераций!
Ура! Уже  70  итераций!
Ура! Уже  80  итераций!
Ура! Уже  90  итераций!
Ура! Уже  100  итераций!
Ура! Уже  110  итераций!
Ура! Уже  120  ите

In [177]:
solutionAnimator1_1 = PDESolutionAnimator(model_name+'1', model_title)
solutionAnimator1_2 = PDESolutionAnimator(model_name+'2', model_title)

In [178]:
x1,y1,t1 = pde1_1.get_mesh(x_cnt,y_cnt,t_cnt)
x2,y2,t2 = pde1_2.get_mesh(x_cnt,y_cnt,t_cnt)

In [ ]:
solutionAnimator1_1.get_cmap_gif_v2(explicit_solution1_1,outFPS,'cmap1_1_ex.gif')
solutionAnimator1_1.get_cmap_gif_v2(implicit_solution1_1,outFPS,'cmap1_1_im.gif')
solutionAnimator1_2.get_cmap_gif_v2(explicit_solution1_2,outFPS,'cmap1_2_ex.gif')
solutionAnimator1_2.get_cmap_gif_v2(implicit_solution1_2,outFPS,'cmap1_2_im.gif')

solutionAnimator1_1.get_surface_gif_v1(explicit_solution1_1,x1[0],y1[0],outFPS,'surface1_1_ex.gif')
solutionAnimator1_1.get_surface_gif_v1(implicit_solution1_1,x1[0],y1[0],outFPS,'surface1_1_im.gif')
solutionAnimator1_2.get_surface_gif_v1(explicit_solution1_2,x2[0],y2[0],outFPS,'surface1_2_ex.gif')
solutionAnimator1_2.get_surface_gif_v1(implicit_solution1_2,x2[0],y2[0],outFPS,'surface1_2_im.gif')

solutionAnimator1_1.comparation_mp4(implicit_solution1_1,explicit_solution1_1,x1[0],y1[0],t1,outFPS,'comp1_1.mp4')

solutionAnimator1_2.comparation_mp4(implicit_solution1_2,explicit_solution1_2,x2[0],y2[0],t2,outFPS,'comp1_2.mp4')


## Модель 2

$$\begin{align}
& f=0 \\
&v_1 = x^10(1-x)y(1-y) \\
&v_2 = 0\\
&g = 0
\end{align}$$

In [ ]:
f = lambda x,y,t: np.zeros(x.shape)
g0 = lambda x,y,t: np.zeros(x.shape)
g1 = lambda x,y,t: np.zeros(x.shape)
g2 = lambda x,y,t: np.zeros(x.shape)
g3 = lambda x,y,t: np.zeros(x.shape)
v_1 = lambda x,y: x**10*(1-x)*y*(1-y)
v_2 = lambda x,y: np.zeros(x.shape)

x_bord = np.array([0,1]);  y_bord = np.array([0,1]);  t1_bord = np.array([0,1]); t2_bord = np.array([0,1]);
x_cnt = 100; y_cnt = 100; t_cnt = 300;
model_name = 'model_2'
model_title = r'$\begin{array}& g = 0, & v_1 = x^{10}(1-x)y(1-y), & v_2 = 0, & f = 0\end{array}$'

In [ ]:
pde2_1 = PDE(f,v_1,v_2,g0,g1,g2,g3,x_bord,y_bord,t1_bord)
pde2_2 = PDE(f,v_1,v_2,g0,g1,g2,g3,x_bord,y_bord,t2_bord)

In [ ]:
explicit_solution2_1 = pde2_1.get_explicit_solution(x_cnt,y_cnt,t_cnt)
implicit_solution2_1 = pde2_1.get_implicit_solution(x_cnt,y_cnt,t_cnt)

explicit_solution2_2 = pde2_2.get_explicit_solution(x_cnt,y_cnt,t_cnt)
implicit_solution2_2 = pde2_2.get_implicit_solution(x_cnt,y_cnt,t_cnt)

In [ ]:
solutionAnimator2_1 = PDESolutionAnimator(model_name+'1', model_title)
solutionAnimator2_2 = PDESolutionAnimator(model_name+'2', model_title)

In [ ]:
x1,y1,t1 = pde1_1.get_mesh(x_cnt,y_cnt,t_cnt)
x2,y2,t2 = pde1_2.get_mesh(x_cnt,y_cnt,t_cnt)

In [ ]:
solutionAnimator2_1.get_cmap_gif_v2(explicit_solution2_1,outFPS,'cmap2_1_ex.gif')
solutionAnimator2_1.get_cmap_gif_v2(implicit_solution2_1,outFPS,'cmap2_1_im.gif')
solutionAnimator2_2.get_cmap_gif_v2(explicit_solution2_2,outFPS,'cmap2_2_ex.gif')
solutionAnimator2_2.get_cmap_gif_v2(implicit_solution2_2,outFPS,'cmap2_2_im.gif')

solutionAnimator2_1.get_surface_gif_v1(explicit_solution2_1,x1[0],y1[0],outFPS,'surface2_1_ex.gif')
solutionAnimator2_1.get_surface_gif_v1(implicit_solution2_1,x1[0],y1[0],outFPS,'surface2_1_im.gif')
solutionAnimator2_2.get_surface_gif_v1(explicit_solution2_2,x2[0],y2[0],outFPS,'surface2_2_ex.gif')
solutionAnimator2_2.get_surface_gif_v1(implicit_solution2_2,x2[0],y2[0],outFPS,'surface2_2_im.gif')

solutionAnimator2_1.comparation_mp4(implicit_solution2_1,explicit_solution2_1,x1[0],y1[0],t1,outFPS,'comp2_1.mp4')

solutionAnimator2_2.comparation_mp4(implicit_solution2_2,explicit_solution2_2,x2[0],y2[0],t2,outFPS,'comp2_2.mp4')

## Модель 3

$$\begin{align}
& f = 0\\
& v_1 = 0\\
& v_2 = x^10(1-x)y(1-y)\\
& g = 0
\end{align}$$


In [ ]:
f = lambda x,y,t: np.zeros(x.shape)
g0 = lambda x,y,t: np.zeros(x.shape)
g1 = lambda x,y,t: np.zeros(x.shape)
g2 = lambda x,y,t: np.zeros(x.shape)
g3 = lambda x,y,t: np.zeros(x.shape)
v_1 = lambda x,y: np.zeros(x.shape)
v_2 = lambda x,y: x**10*(1-x)*y*(1-y)

x_bord = np.array([0,1]);  y_bord = np.array([0,1]);  t1_bord = np.array([0,1]); t2_bord = np.array([0,1]);
x_cnt = 100; y_cnt = 100; t_cnt = 300;
model_name = 'model_3'
model_title = r'$\begin{array}& g = 0, & v_1 = 0, & v_2 = x^{10}(1-x)y(1-y), & f = 0\end{array}$'

In [ ]:
pde3_1 = PDE(f,v_1,v_2,g0,g1,g2,g3,x_bord,y_bord,t1_bord)
pde3_2 = PDE(f,v_1,v_2,g0,g1,g2,g3,x_bord,y_bord,t2_bord)

In [ ]:
explicit_solution3_1 = pde3_1.get_explicit_solution(x_cnt,y_cnt,t_cnt)
implicit_solution3_1 = pde3_1.get_implicit_solution(x_cnt,y_cnt,t_cnt)

explicit_solution3_2 = pde3_2.get_explicit_solution(x_cnt,y_cnt,t_cnt)
implicit_solution3_2 = pde3_2.get_implicit_solution(x_cnt,y_cnt,t_cnt)

In [ ]:
solutionAnimator3_1 = PDESolutionAnimator(model_name+'1', model_title)
solutionAnimator3_2 = PDESolutionAnimator(model_name+'2', model_title)

In [ ]:
x1,y1,t1 = pde1_1.get_mesh(x_cnt,y_cnt,t_cnt)
x2,y2,t2 = pde1_2.get_mesh(x_cnt,y_cnt,t_cnt)

In [ ]:
solutionAnimator3_1.get_cmap_gif_v2(explicit_solution3_1,outFPS,'cmap3_1_ex.gif')
solutionAnimator3_1.get_cmap_gif_v2(implicit_solution3_1,outFPS,'cmap3_1_im.gif')
solutionAnimator3_2.get_cmap_gif_v2(explicit_solution3_2,outFPS,'cmap3_2_ex.gif')
solutionAnimator3_2.get_cmap_gif_v2(implicit_solution3_2,outFPS,'cmap3_2_im.gif')

solutionAnimator3_1.get_surface_gif_v1(explicit_solution3_1,x1[0],y1[0],outFPS,'surface3_1_ex.gif')
solutionAnimator3_1.get_surface_gif_v1(implicit_solution3_1,x1[0],y1[0],outFPS,'surface3_1_im.gif')
solutionAnimator3_2.get_surface_gif_v1(explicit_solution3_2,x2[0],y2[0],outFPS,'surface3_2_ex.gif')
solutionAnimator3_2.get_surface_gif_v1(implicit_solution3_2,x2[0],y2[0],outFPS,'surface3_2_im.gif')

solutionAnimator3_1.comparation_mp4(implicit_solution3_1,explicit_solution3_1,x1[0],y1[0],t1,outFPS,'comp3_1.mp4')

solutionAnimator3_2.comparation_mp4(implicit_solution3_2,explicit_solution3_2,x2[0],y2[0],t2,outFPS,'comp3_2.mp4')


## Модель 4

$$\begin{align}
& f = 0\\
& v_1 = (1−x)y(1−y)\cos{(5\pi x)}\\
& v_2 = 0\\
& g(0, y) = y(1 − y), g(1, y) = 0, g(x, 0) = 0, g(x, 1) = 0
\end{align}$$

In [ ]:
f = lambda x,y,t: np.zeros(x.shape)
g0 = lambda x,y,t: y*(1-y)
g1 = lambda x,y,t: np.zeros(x.shape)
g2 = lambda x,y,t: np.zeros(x.shape)
g3 = lambda x,y,t: np.zeros(x.shape)
v_1 = lambda x,y: (1-x)*y*(1-y)*np.cos(5*np.pi*x)
v_2 = lambda x,y: np.zeros(x.shape)

x_bord = np.array([0,1]);  y_bord = np.array([0,1]);  t1_bord = np.array([0,1]); t2_bord = np.array([0,1]);
x_cnt = 100; y_cnt = 100; t_cnt = 300;
model_name = 'model_4'
model_title = r'$\begin{array}& g(0,y) = y(1-y), & g(1,y) = 0, & g(x,0) = 0, & g(x,1) = 0, & v_1= (1-x)y(1-y)\cos(5\pi x), & v_2 = 0, & f = 0 \end{array}$'

In [ ]:
pde4_1 = PDE(f,v_1,v_2,g0,g1,g2,g3,x_bord,y_bord,t1_bord)
pde4_2 = PDE(f,v_1,v_2,g0,g1,g2,g3,x_bord,y_bord,t2_bord)

In [ ]:
explicit_solution4_1 = pde4_1.get_explicit_solution(x_cnt,y_cnt,t_cnt)
implicit_solution4_1 = pde4_1.get_implicit_solution(x_cnt,y_cnt,t_cnt)

explicit_solution4_2 = pde4_2.get_explicit_solution(x_cnt,y_cnt,t_cnt)
implicit_solution4_2 = pde4_2.get_implicit_solution(x_cnt,y_cnt,t_cnt)

In [ ]:
solutionAnimator4_1 = PDESolutionAnimator(model_name+'1', model_title)
solutionAnimator4_2 = PDESolutionAnimator(model_name+'2', model_title)

In [ ]:
x1,y1,t1 = pde1_1.get_mesh(x_cnt,y_cnt,t_cnt)
x2,y2,t2 = pde1_2.get_mesh(x_cnt,y_cnt,t_cnt)

In [ ]:
solutionAnimator4_1.get_cmap_gif_v2(explicit_solution4_1,outFPS,'cmap4_1_ex.gif')
solutionAnimator4_1.get_cmap_gif_v2(implicit_solution4_1,outFPS,'cmap4_1_im.gif')
solutionAnimator4_2.get_cmap_gif_v2(explicit_solution4_2,outFPS,'cmap4_2_ex.gif')
solutionAnimator4_2.get_cmap_gif_v2(implicit_solution4_2,outFPS,'cmap4_2_im.gif')

solutionAnimator4_1.get_surface_gif_v1(explicit_solution4_1,x1[0],y1[0],outFPS,'surface4_1_ex.gif')
solutionAnimator4_1.get_surface_gif_v1(implicit_solution4_1,x1[0],y1[0],outFPS,'surface4_1_im.gif')
solutionAnimator4_2.get_surface_gif_v1(explicit_solution4_2,x2[0],y2[0],outFPS,'surface4_2_ex.gif')
solutionAnimator4_2.get_surface_gif_v1(implicit_solution4_2,x2[0],y2[0],outFPS,'surface4_2_im.gif')

solutionAnimator4_1.comparation_mp4(implicit_solution4_1,explicit_solution4_1,x1[0],y1[0],t1,outFPS,'comp4_1.mp4')

solutionAnimator4_2.comparation_mp4(implicit_solution4_2,explicit_solution4_2,x2[0],y2[0],t2,outFPS,'comp4_2.mp4')


## Модель 5

$$\begin{align}
& f = 0\\
& v_1 = 0\\
& v_2 = 0\\
& g = \dfrac{5\sin{t}}{t+1}
\end{align}$$


In [ ]:
f = lambda x,y,t: np.zeros(x.shape)
g0 = lambda x,y,t: 5*np.sin(t)/(t+1)
g1 = lambda x,y,t: 5*np.sin(t)/(t+1)
g2 = lambda x,y,t: 5*np.sin(t)/(t+1)
g3 = lambda x,y,t: 5*np.sin(t)/(t+1)
v_1 = lambda x,y: np.zeros(x.shape)
v_2 = lambda x,y: np.zeros(x.shape)

x_bord = np.array([0,1]);  y_bord = np.array([0,1]);  t1_bord = np.array([0,1]); t2_bord = np.array([0,1]);
x_cnt = 100; y_cnt = 100; t_cnt = 300;
model_name = 'model_5'
model_title = r'$\begin{array}& g = \dfrac{5\sin{t}}{t+1}, & v_1 = 0, &  v_2 = 0, & f = 0 \end{array}$'

In [ ]:
pde5_1 = PDE(f,v_1,v_2,g0,g1,g2,g3,x_bord,y_bord,t1_bord)
pde5_2 = PDE(f,v_1,v_2,g0,g1,g2,g3,x_bord,y_bord,t2_bord)

In [ ]:
explicit_solution5_1 = pde5_1.get_explicit_solution(x_cnt,y_cnt,t_cnt)
implicit_solution5_1 = pde5_1.get_implicit_solution(x_cnt,y_cnt,t_cnt)

explicit_solution5_2 = pde5_2.get_explicit_solution(x_cnt,y_cnt,t_cnt)
implicit_solution5_2 = pde5_2.get_implicit_solution(x_cnt,y_cnt,t_cnt)

In [ ]:
solutionAnimator5_1 = PDESolutionAnimator(model_name+'1', model_title)
solutionAnimator5_2 = PDESolutionAnimator(model_name+'2', model_title)

In [ ]:
x1,y1,t1 = pde1_1.get_mesh(x_cnt,y_cnt,t_cnt)
x2,y2,t2 = pde1_2.get_mesh(x_cnt,y_cnt,t_cnt)

In [ ]:
solutionAnimator5_1.get_cmap_gif_v2(explicit_solution5_1,outFPS,'cmap5_1_ex.gif')
solutionAnimator5_1.get_cmap_gif_v2(implicit_solution5_1,outFPS,'cmap5_1_im.gif')
solutionAnimator5_2.get_cmap_gif_v2(explicit_solution5_2,outFPS,'cmap5_2_ex.gif')
solutionAnimator5_2.get_cmap_gif_v2(implicit_solution5_2,outFPS,'cmap5_2_im.gif')

solutionAnimator5_1.get_surface_gif_v1(explicit_solution5_1,x1[0],y1[0],outFPS,'surface5_1_ex.gif')
solutionAnimator5_1.get_surface_gif_v1(implicit_solution5_1,x1[0],y1[0],outFPS,'surface5_1_im.gif')
solutionAnimator5_2.get_surface_gif_v1(explicit_solution5_2,x2[0],y2[0],outFPS,'surface5_2_ex.gif')
solutionAnimator5_2.get_surface_gif_v1(implicit_solution5_2,x2[0],y2[0],outFPS,'surface5_2_im.gif')

solutionAnimator5_1.comparation_mp4(implicit_solution5_1,explicit_solution5_1,x1[0],y1[0],t1,outFPS,'comp5_1.mp4')

solutionAnimator5_2.comparation_mp4(implicit_solution5_2,explicit_solution5_2,x2[0],y2[0],t2,outFPS,'comp5_2.mp4')
